In [21]:
import os

In [29]:
%pwd

'/home/aman/Desktop/Cognitext'

In [28]:
os.chdir("Cognitext")

In [35]:
from pathlib import Path
from dataclasses import dataclass

In [45]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    data_path: Path
    save_dir: Path

In [37]:
from Cognitext.utils.common import read_yaml, create_directories
from Cognitext.constants import *

In [43]:
class ConfigurationManager:
    def __init__(self, param = PARAMS_FILE_PATH, config = CONFIG_FILE_PATH):
        self.params = read_yaml(param)
        self.config = read_yaml(config)

        create_directories([self.config.artifacts_root])

    def get_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        get_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            data_path = config.data_path,
            save_dir = config.save_dir
        )
        return get_ingestion_config

In [40]:
import os
import urllib.request as request
from Cognitext.logging import logger
from Cognitext.utils.common import get_size
import zipfile

In [47]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    def download_Data(self):
        if not os.path.exists(self.config.data_path):
            filename, headers = request.urlretrieve(url=self.config.source_URL, filename=self.config.data_path)
            logger.info(f"{filename}, Downloaded Sucessfully!")
        else:
            logger.info(f"File already exists of size {get_size(Path(self.config.data_path))}")

    def unzip_data(self):
        unzip_path = self.config.save_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.data_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [50]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_Data()
    data_ingestion.unzip_data()
except Exception as e:
    raise

[2024-09-17 22:31:18,387: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-17 22:31:18,391: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-09-17 22:31:18,394: INFO: common: created directory at: artifacts]
[2024-09-17 22:31:18,397: INFO: common: created directory at: artifacts/data_ingestion]
[2024-09-17 22:31:26,818: INFO: 890897275: artifacts/data_ingestion/Data.zip, Downloaded Sucessfully!]
